## Plano de hoje

- Erros absolutos e relativos
- Método de Newton

# Erros, erros, erros!

Em geral, é muito pouco comum que os métodos computacionais que nós vamos usar dêem uma resposta exata.
Por exemplo, se queremos calcular $\sqrt2$, o computador não poderá jamais dar todas as casas decimais deste número.
Mais ainda, visto que o computador usa representação em base 2, também não será capaz de escrever $0.1$ exatamente!

Mas (para problemas do mundo "real") isto não é um problema.
Afinal, muitas vezes estes problemas vêem com dados sujeitos a erros de medida, e portanto não faz sentido achar uma resposta tão exata quando o problema original já é inexato.
Assim, o mais importante é entender o quão longe da resposta estamos, e (quando possível) **garantir** que não estamos longe demais.

Nesta primeira abordagem, vamos nos concentrar em achar um zero de uma função $f:\mathbf{R}\to \mathbf{R}$.
A intuição que desenvolveremos aqui servirá para os métodos da bisseção e de Newton (e também para resolver equações lineares $Ax = b$), e pode ser facilmente estendida para a análise de derivadas, integrais e outros problemas de que trataremos ao longo do curso.

## Intervalos

Uma forma relativamente simples de garantir uma raiz de $f$ é dizer que _existe_ uma raiz de $f$ num certo intervalo $[a,b]$.
Quanto menor o comprimento do intervalo, mais precisa será a resposta à questão "_Exiba uma raiz de $f$_".

Observe que o método da bisseção já dá esta informação.
Mais ainda, ele permite que, dado um intervalo, possamos "melhorar" cada vez mais a qualidade da resposta, calculando $f$ em mais alguns pontos.

## Erros

Mas o mais comum não é garantir um intervalo, porque (muitas vezes) o método que usamos irá mais naturalmente fornecer um único valor $x$ que está perto da raiz "de verdade" $z$.

Existem duas medidas naturais para o erro: absoluto e relativo.

## Erro absoluto

O erro absoluto é definido como $\lvert x - z\rvert$, ou seja, o quanto a resposta do computador difere do valor "real".

## Erro relativo

O erro relativo é definido como $\displaystyle \left\lvert \frac{x - z}{z} \right\rvert$, ou seja, o quanto, _proporcionalmente_, a resposta do computador difere do valor "real".

### Exercício

Dê exemplos de situação onde

- O erro relativo seja maior do que o erro absoluto.
- O erro absoluto seja maior do que o erro relativo.
- O erro absoluto seja igual ao erro relativo.

### Exercício

Escreva uma função que, dados dois números, retorne o erro absoluto e o erro relativo.

In [1]:
def erros(target, calculated):
    abs_err = abs(target-calculated)
    rel_err = abs( (target-calculated)/calculated )
    
    return abs_err, rel_err

Esta função é simétrica? ($f(a,b) = f(b,a)$?)

$f(a,b) = \lvert a-b \rvert, \displaystyle \left\lvert \frac{a-b}{b} \right\rvert$

$f(b,a) = \lvert b-a \rvert, \displaystyle \left\lvert \frac{b-a}{a} \right\rvert$

Se $f(a,b) = f(b,a)$, então $\lvert a-b \rvert = \lvert b-a \rvert$ e $\displaystyle \left\lvert \frac{a-b}{b} \right\rvert = \left\lvert \frac{b-a}{a} \right\rvert$

A primeira igualdade é verdadeira para quaisquer $a$ e $b$

Da segunda decorre que:

$\displaystyle \left\lvert \frac{1}{a} \right\rvert = \left\lvert \frac{1}{b} \right\rvert$

$\lvert a \rvert = \lvert b \rvert$

A segunda igualdade é verdadeira só se $\lvert a \rvert = \lvert b \rvert$, então $f(a,b)$ não é simétrica.

# O método de Newton para calcular raízes

O método da bisseção é bastante geral (funciona para qualquer função contínua!),
e converge "geométricamente rápido": o erro na etapa $n+1$ será, aproximadamente,
a metade do erro da etapa anterior.

Para funções cuja derivada é conhecida, entretanto,
o _método de Newton_ é uma alternativa muito poderosa,
pois converge com maior velocidade.
Além disso, ele dispensa conhecer dois pontos onde o sinal da função seja diferente.
Vejamos como ele funciona.

## Idéia geométrica

Dado um ponto $(x,f(x))$ no gráfico de $f$, se traçarmos a tangente,
esta será uma boa aproximação da função "perto" de $x$.
Assim, seguimos esta reta tangente até que ela encontre o eixo-$x$ no ponto $(z,0)$,
esperando que esta interseção esteja próxima da verdadeira raiz,
que é a interseção da _curva_ descrita por $f$ e o eixo-$x$.

Em fórmulas, temos:
$$ (z,0) \in T = \big\{ (x, f(x)) + t (1, f'(x)) \mid t \in R \big\} $$
para o ponto $(z,0)$ que está na reta tangente $T$ e também no eixo-$x$
(pois sua coordenada $y = 0$).
Resolvendo o sistema, encontramos
$$ z = x - \frac{f(x)}{f'(x)}. $$

A presença de $f'(x)$ no denominador mostra que este método funciona **mal**
quando está próximo de uma raiz de $f'$.
Além disso, o método de Newton não fornece um "intervalo de confiança" como no caso da bisseção.

Assim, é muito importante ter aqui um critério de convergência para poder parar as iterações.
Em geral, este pode ser dado por três diferentes parâmetros:

- O número de iterações feitas: se estamos calculando "há muito tempo", talvez o método esteja "perdido"
- A distância de $f(x)$ para zero: talvez já tenhamos calculado algo suficientemente próximo de uma raiz,
    se $\lvert f(x)\rvert << 1$
- A distância de $x$ para um zero: se a diferença entre dois pontos sucessivos ($x$ e $z$ no nosso exemplo)
    for pequena, então é _provável_ que estejamos perto de uma raiz.

## Impementação

Para programar o método de Newton como uma função recursiva,
podemos fazer um paralelo com o método da bisseção.
No caso da bisseção, a cada etapa testávamos se a aproximação já estava suficientemente perto
(por exemplo, se uma estimativa do erro absoluto fosse pequena),
e caso contrário dividíamos o intervalo por 2 para continuar buscando a raiz.

Aqui, também vamos estimar o erro (só que desta vez não há mais _garantia_ de que o erro será menor do que a estimativa),
e, se este ainda for "grande", vamos produzir um novo ponto (usando a tangente) para continuar buscando uma raiz.

In [2]:
def newton(f,df,x, prec=1e-8, maxiter=100):
    if maxiter == 0:
        return None
    dx = f(x)/df(x)
    newx = x - dx
    
    if abs(dx) < prec:
        return newx
    else:
        return newton(f,df,newx, prec,maxiter-1)

## Usando `print` e `format`

Vamos escrever uma função para nos ajudar a fazer os testes do método de Newton, e comparar com a bisseção.
Como desejamos comparar os valores retornados por ambos os métodos,
é importante que estes sejam fáceis de ler na tela.

O mecanismo do `Out[]` do IPython (onde bastaria que retornássemos alguns valores) é bastante útil,
mas obriga a lembrar todo o contexto.
Com `print`, podemos incluir informações textuais a mais,
além de formatar os valores de maneira uniforme (usando `.format()` ou `%`),
o que ajuda a comparação.

- Exemplos: de usos [mais comuns][mkaz], e os [oficiais do Python][py-ex]
- Referência: [a documentação do Python][doc]

[py-ex]: https://docs.python.org/3/library/string.html#format-examples
[doc]:   https://docs.python.org/3/library/string.html#format-string-syntax
[mkaz]:  https://mkaz.github.io/2012/10/10/python-string-format/

In [3]:
def testar(f,df,a,b):
    xb = bissecao(f,a,b, prec=1e-10)
    xn = newton(f,df,a,  prec=1e-10)
    print('''\
Bisseção: z ~= {: 18.10e} (f(z) = {: .8f})
Newton  : z ~= {: 18.10e} (f(z) = {: .8f})'''.format(xb,f(xb), xn,f(xn)))

In [4]:
def bissecao(f,a,b,prec):
    """ Método da bisseção para uma função f no intervalo [a,b]. """
    m = (a+b)/2
    # Se já há precisão suficiente, retornamos o ponto médio do intervalo
    if abs(b - a) < prec/2: return m
    # Se f(m) == 0, achamos uma raiz exata!
    if f(m) == 0: return m

    # Senão, vamos por recorrência
    if f(m)*f(a) < 0: return bissecao(f,a,m,prec)
    else: return bissecao(f,m,b,prec)

In [5]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
testar(sin,cos,1,4)

Bisseção: z ~=   3.1415926536e+00 (f(z) = -0.00000000)
Newton  : z ~=   0.0000000000e+00 (f(z) =  0.00000000)


In [7]:
def f(x): return x**3 - 2
def df(x): return 3*x**2

In [8]:
testar(f,df,2.5,0)

Bisseção: z ~=   1.2599210499e+00 (f(z) = -0.00000000)
Newton  : z ~=   1.2599210499e+00 (f(z) =  0.00000000)


### Exercício

Modifique os métodos da bisseção e de Newton para que eles retornem também o número total de vezes que a função "entrou" na recursão, e use essa informação numa nova função `testar`.

In [9]:
def newton(f,df,x, prec=1e-8, maxiter=100):
    if maxiter == 0:
        return None
    dx = f(x)/df(x)
    newx = x - dx
    
    if abs(dx) < prec:
        return newx
    else:
        return newton(f,df,newx, prec,maxiter-1)

In [10]:
def bissecao(f,a,b,prec):
    """ Método da bisseção para uma função f no intervalo [a,b]. """
    m = (a+b)/2
    # Se já há precisão suficiente, retornamos o ponto médio do intervalo
    if abs(b - a) < prec/2: return m
    # Se f(m) == 0, achamos uma raiz exata!
    if f(m) == 0: return m

    # Senão, vamos por recorrência
    if f(m)*f(a) < 0:
        return bissecao(f,a,m,prec)
    else:
        return bissecao(f,m,b,prec)

In [11]:
def testar(f,df,a,b):
    xb = bissecao(f,a,b, prec=1e-10)
    xn = newton(f,df,a,  prec=1e-10)
    print('''\
Bisseção: z ~= {: 18.10e} (f(z) = {: .8f})
Newton  : z ~= {: 18.10e} (f(z) = {: .8f})'''.format(xb,f(xb), xn,f(xn)))

In [12]:
testar(sin,cos,4,1)

Bisseção: z ~=   3.1415926536e+00 (f(z) = -0.00000000)
Newton  : z ~=   3.1415926536e+00 (f(z) =  0.00000000)


In [13]:
testar(f,df,2.5,0)

Bisseção: z ~=   1.2599210499e+00 (f(z) = -0.00000000)
Newton  : z ~=   1.2599210499e+00 (f(z) =  0.00000000)
